In [ ]:
!pip install -q bertopic
!pip install -q sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.0 MB/s eta 0:00:00


In [ ]:
!pip install -q keybert transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.3 MB/s eta 0:00:00


In [ ]:
import zipfile
import os
import pandas as pd
import os
import zipfile
import json
from keybert import KeyBERT
from google.colab import files

In [ ]:
print("Upload your ZIP files (each containing .txt lecture transcripts):")
uploaded = files.upload()

Upload your ZIP files (each containing .txt lecture transcripts):


Saving MIT_LINEAR_ALGEBRA.zip to MIT_LINEAR_ALGEBRA.zip
Saving DEEP_LEARNING_TRANSCRIPTS.zip to DEEP_LEARNING_TRANSCRIPTS.zip
Saving DATA_STRUCTURES&ALGORITHMS.zip to DATA_STRUCTURES&ALGORITHMS.zip
Saving MACHINE_LEARNING_CALTECH.zip to MACHINE_LEARNING_CALTECH.zip
Saving MIT_ARTIFICIAL_INTELLIGENCE.zip to MIT_ARTIFICIAL_INTELLIGENCE.zip


In [ ]:
kw_model = KeyBERT("all-MiniLM-L6-v2")
TOPIC_DELIMITER = " | "

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

FOR JSONL STORAGE ALL LECTURES IN ONE FILE

In [ ]:
for zip_filename in uploaded:
    base_name = os.path.splitext(zip_filename)[0]
    extract_path = os.path.join("/content", base_name)


    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall(extract_path)


    txt_files = []
    for root, _, files_in_dir in os.walk(extract_path):
        for file in files_in_dir:
            if file.endswith(".txt"):
                txt_files.append(os.path.join(root, file))

    if not txt_files:
        print(f"No .txt files found in {zip_filename}")
        continue

    # Write to JSONL
    jsonl_path = os.path.join("/content", base_name + ".jsonl")
    with open(jsonl_path, "w", encoding="utf-8") as out_file:
        for file_path in sorted(txt_files):
            with open(file_path, "r", encoding="utf-8") as f:
                raw_text = f.read().strip()
                transcript = " ".join(raw_text.split())

            if not transcript:
                print(f"Skipping empty file: {file_path}")
                continue

            extracted = kw_model.extract_keywords(
                transcript,
                keyphrase_ngram_range=(1, 1),
                stop_words='english',
                top_n=15
            )

            if not extracted:
                continue

            topics = [kw for kw, _ in extracted]
            scores = [round(score, 4) for _, score in extracted]
            title = os.path.basename(file_path).replace(".txt", "")

            # Final
            block = f"""### Instruction:
Generate relevant content based on the following:
Title: {title}
Topics: {topics}
Topic Weights: {scores}

### Response:
{transcript}
"""

            out_file.write(block.strip() + "\n")

    print(f"Saved JSONL: {jsonl_path}")


Saved JSONL: /content/MACHINE_LEARNING_CALTECH.jsonl
Saved JSONL: /content/DEEP_LEARNING_TRANSCRIPTS.jsonl
Saved JSONL: /content/DATA_STRUCTURES&ALGORITHMS.jsonl


In [ ]:
def add_keywords(ngram_range, top_n):
                extracted = kw_model.extract_keywords(
                    transcript,
                    keyphrase_ngram_range=(ngram_range, ngram_range),
                    stop_words='english',
                    top_n=top_n
                )
                for kw, score in extracted:
                    keywords_dict[kw] = round(score, 4)

JSONL text key format with duplicated keywords tackled

In [ ]:
for zip_filename in uploaded:
    base_name = os.path.splitext(zip_filename)[0]
    extract_path = os.path.join("/content", base_name)

    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

    txt_files = []
    for root, _, files_in_dir in os.walk(extract_path):
        for file in files_in_dir:
            if file.endswith(".txt"):
                txt_files.append(os.path.join(root, file))

    if not txt_files:
        print(f"No .txt files found in {zip_filename}")
        continue


    jsonl_output_path = os.path.join("/content", f"{base_name}.jsonl")
    with open(jsonl_output_path, "w", encoding="utf-8") as out_file:

        for file_path in sorted(txt_files):
            with open(file_path, "r", encoding="utf-8") as f:
                raw_text = f.read().strip()
                transcript = " ".join(raw_text.split())

            if not transcript:
                print(f"Skipping empty file: {file_path}")
                continue

            keywords_dict = {}

            add_keywords(1, 10)
            add_keywords(2, 10)
            add_keywords(3, 5)

            final_keywords = {}
            for phrase, score in sorted(keywords_dict.items(), key=lambda x: -x[1]):
                if not any(word in final_keywords for word in phrase.split()):
                    final_keywords[phrase] = score

            title = os.path.basename(file_path).replace(".txt", "")
            topics = list(final_keywords.keys())
            weights = list(final_keywords.values())

            # Format the JSONL line
            text_block = f"""### Instruction:
Generate content using the following:

Title: {title}
Topics: {', '.join(topics)}
Topic Weights: {', '.join(map(str, weights))}

### Response:
{transcript}"""

            out_file.write(json.dumps({"text": text_block}, ensure_ascii=False) + "\n")

    print(f"Created JSONL file: {jsonl_output_path}")


Created JSONL file: /content/MACHINE_LEARNING_CALTECH.jsonl
Created JSONL file: /content/DEEP_LEARNING_TRANSCRIPTS.jsonl
Created JSONL file: /content/DATA_STRUCTURES&ALGORITHMS.jsonl


FOR CSV BASED STORAGE

In [ ]:
for zip_filename in uploaded:
    base_name = os.path.splitext(zip_filename)[0]
    extract_path = os.path.join("/content", base_name)

    print(f"Extracting: {zip_filename} -> {extract_path}")

    # Extract ZIP
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

    # Recursively collect all .txt files inside extracted folder
    txt_files = []
    for root, dirs, files_in_dir in os.walk(extract_path):
        for file in files_in_dir:
            if file.endswith(".txt"):
                txt_files.append(os.path.join(root, file))

    if not txt_files:
        print(f"No .txt files found in {zip_filename}")
        continue

    print(f"Found {len(txt_files)} .txt files in {zip_filename}")

    # Extract topics from .txt files
    records = []
    lecture_no = 1

    for file_path in sorted(txt_files):
        with open(file_path, "r", encoding="utf-8") as f:
            text = f.read().strip()

        if not text:
            print(f"Skipping empty file: {file_path}")
            continue

        extracted = kw_model.extract_keywords(
            text,
            keyphrase_ngram_range=(1, 3),
            stop_words='english',
            top_n=7
        )
        clean_text = " ".join(text.strip().split())
        topic_score_dict = {kw: round(score, 4) for kw, score in extracted}

        records.append({
            "lecture_no": lecture_no,
            "lecture_title": os.path.basename(file_path).replace(".txt", ""),
            "transcript": clean_text,
            "topic_with_scores": str(topic_score_dict)
        })

        lecture_no += 1

    # Save to CSV

    output_csv = base_name + "_topics.csv"
    import csv
    df = pd.DataFrame(records)
    df.to_csv(output_csv, index=False, quoting=csv.QUOTE_ALL)
    df = pd.DataFrame(records)
    print(f"Saved CSV: {output_csv}")

Extracting: DEEP_LEARNING_TRANSCRIPTS.zip -> /content/DEEP_LEARNING_TRANSCRIPTS
Found 111 .txt files in DEEP_LEARNING_TRANSCRIPTS.zip
Saved CSV: DEEP_LEARNING_TRANSCRIPTS_topics.csv


In [ ]:
df

,lecture_no,lecture_title,transcript,topic_with_scores
0,1,Deep Learning - Course Introduction,"Hi, I am Vin. So, this is introduction to the ...","{'neural network uses': 0.5954, 'use deep lear..."
1,2,Deep Learning(CS7015)： Lec 1.1 Biological Neuron,"Hello everyone, welcome to lecture 1 of CS7015...","{'known neuron doctrine': 0.52, 'neuron doctri..."
2,3,Deep Learning(CS7015)： Lec 1.2 From Spring to ...,And now what we will get into in the next chap...,"{'ai considered': 0.5243, 'ai going': 0.5228, ..."
3,4,Deep Learning(CS7015)： Lec 1.3 The Deep Revival,"When this deep revival happened right. So, in ...","{'neural network did': 0.495, 'make deep neura..."
4,5,Deep Learning(CS7015)： Lec 1.4 From Cats to Co...,I will talk about the history of convolutional...,"{'experiments various neural': 0.5566, 'stimul..."
...,...,...,...,...
106,107,Deep Learning(CS7015)： Lec 9.1 A quick recap o...,Welcome to lecture 9 of CS7015. Today we will ...,"{'train deep neural': 0.5742, 'taught gradient..."
107,108,Deep Learning(CS7015)： Lec 9.2 Unsupervised pr...,"So, with that we go on to the next module in w...","{'unsupervised pre training': 0.6233, 'learnin..."
108,109,Deep Learning(CS7015)： Lec 9.3 Better activati...,"So, let us start with better activation functi...","{'better activation functions': 0.6646, 'activ..."
109,110,Deep Learning(CS7015)： Lec 9.4 Better initiali...,"So, in this module we will talk about better i...","{'deep learning evolved': 0.491, 'deep neural'..."


ALL IN SINGLE FILE

In [ ]:
# Output file (single)
train_jsonl_path = "/content/train.jsonl"
with open(train_jsonl_path, "w", encoding="utf-8") as out_file:

    for zip_filename in uploaded:
        base_name = os.path.splitext(zip_filename)[0]
        extract_path = os.path.join("/content", base_name)

        # Unzip
        with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
            zip_ref.extractall(extract_path)

        # Find all .txt files
        txt_files = []
        for root, _, files_in_dir in os.walk(extract_path):
            for file in files_in_dir:
                if file.endswith(".txt"):
                    txt_files.append(os.path.join(root, file))

        if not txt_files:
            print(f"No .txt files found in {zip_filename}")
            continue

        for file_path in sorted(txt_files):
            with open(file_path, "r", encoding="utf-8") as f:
                raw_text = f.read().strip()
                transcript = " ".join(raw_text.split())  # Clean newlines and excess whitespace

            if not transcript:
                print(f"Skipping empty file: {file_path}")
                continue

            # Extract multi-gram keywords
            keywords_dict = {}

            def add_keywords(ngram_range, top_n):
                extracted = kw_model.extract_keywords(
                    transcript,
                    keyphrase_ngram_range=(ngram_range, ngram_range),
                    stop_words='english',
                    top_n=top_n
                )
                for kw, score in extracted:
                    keywords_dict[kw] = round(score, 4)

            add_keywords(1, 15)
            add_keywords(2, 10)
            add_keywords(3, 5)

            # Deduplicate: keep longer phrases, drop subparts
            final_keywords = {}
            for phrase, score in sorted(keywords_dict.items(), key=lambda x: -x[1]):
                if not any(word in final_keywords for word in phrase.split()):
                    final_keywords[phrase] = score

            title = os.path.basename(file_path).replace(".txt", "")
            topics = list(final_keywords.keys())
            weights = list(final_keywords.values())

            # Build final JSONL string
            text_block = f"""### Instruction:
Generate content using the following:

Title: {title}
Topics: {', '.join(topics)}
Topic Weights: {', '.join(map(str, weights))}

### Response:
{transcript}"""

            out_file.write(json.dumps({"text": text_block}, ensure_ascii=False) + "\n")

print(f"✅ Final unified JSONL file created: {train_jsonl_path}")


✅ Final unified JSONL file created: /content/train.jsonl


In [ ]:
NEW JSONL FORMAT

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Load embedding model once
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

def extract_keywords(transcript):
    word_count = len(transcript.split())

    # Select number of keywords
    if word_count > 5000:
        keyword_plan = [(1, 10), (2, 5)]  # (ngram, count)
    else:
        keyword_plan = [(1, 6), (2, 4)]

    unique_keywords = []
    keyword_embeddings = []

    def is_semantically_unique(keyword, embeddings, threshold=0.8):
        new_emb = embedding_model.encode([keyword])
        if not embeddings:
            return True
        sim_scores = cosine_similarity(new_emb, embeddings)
        return sim_scores.max() < threshold

    for ngram_range, top_n in keyword_plan:
        try:
            extracted = kw_model.extract_keywords(
                transcript,
                keyphrase_ngram_range=(ngram_range, ngram_range),
                stop_words="english",
                top_n=top_n * 2  # extract more to allow filtering
            )
            for kw, _ in extracted:
                if is_semantically_unique(kw, keyword_embeddings):
                    unique_keywords.append(kw)
                    keyword_embeddings.append(embedding_model.encode([kw])[0])
                    if len([k for k in unique_keywords if len(k.split()) == ngram_range]) >= top_n:
                        break
        except Exception as e:
            print(f"⚠️ Error extracting keywords for ngram {ngram_range}: {e}")

    return ", ".join(unique_keywords)

In [ ]:

# Final single JSONL output file
train_jsonl_path = "/content/train.jsonl"
with open(train_jsonl_path, "w", encoding="utf-8") as out_file:

    for zip_filename in uploaded:
        base_name = os.path.splitext(zip_filename)[0]
        extract_path = os.path.join("/content", base_name)

        # Unzip the ZIP
        with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
            zip_ref.extractall(extract_path)

        # Get all .txt files inside the folder (recursive)
        txt_files = []
        for root, _, files_in_dir in os.walk(extract_path):
            for file in files_in_dir:
                if file.endswith(".txt"):
                    txt_files.append(os.path.join(root, file))

        if not txt_files:
            print(f"No .txt files found in {zip_filename}")
            continue

        for file_path in sorted(txt_files):
            with open(file_path, "r", encoding="utf-8") as f:
                raw_text = f.read().strip()
                transcript = " ".join(raw_text.split())

            if not transcript:
                print(f"Skipping empty file: {file_path}")
                continue

            # Extract keywords of various n-gram sizes
            keywords_dict = {}

            def add_keywords(ngram_range, top_n):
                extracted = kw_model.extract_keywords(
                    transcript,
                    keyphrase_ngram_range=(ngram_range, ngram_range),
                    stop_words='english',
                    top_n=top_n
                )
                for kw, score in extracted:
                    keywords_dict[kw] = round(score, 4)

            add_keywords(1, 15)
            add_keywords(2, 10)
            add_keywords(3, 5)

            # Deduplicate: prefer longer phrases
            final_keywords = {}
            for phrase, score in sorted(keywords_dict.items(), key=lambda x: -x[1]):
                if not any(word in final_keywords for word in phrase.split()):
                    final_keywords[phrase] = score

            title = os.path.basename(file_path).replace(".txt", "")
            topics = list(final_keywords.keys())

            prompt = f"Title: {title}, Keywords: {', '.join(topics)}"
            response = transcript

            json_obj = {
                "Prompt": prompt,
                "Response": response
            }

            out_file.write(json.dumps(json_obj, ensure_ascii=False) + "\n")

print(f"Done. Unified JSONL file saved at: {train_jsonl_path}")

Done. Unified JSONL file saved at: /content/train.jsonl


Prompt Response format

In [ ]:
# Final output file
train_jsonl_path = "/content/train3.jsonl"
with open(train_jsonl_path, "w", encoding="utf-8") as out_file:

    for zip_filename in uploaded:
        base_name = os.path.splitext(zip_filename)[0]
        extract_path = os.path.join("/content", base_name)

        # Unzip
        with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
            zip_ref.extractall(extract_path)

        # Find all .txt files inside ZIP (recursively)
        txt_files = []
        for root, _, files_in_dir in os.walk(extract_path):
            for file in files_in_dir:
                if file.endswith(".txt"):
                    txt_files.append(os.path.join(root, file))

        if not txt_files:
            print(f"No .txt files found in {zip_filename}")
            continue

        for file_path in sorted(txt_files):
            with open(file_path, "r", encoding="utf-8") as f:
                raw_text = f.read().strip()
                transcript = " ".join(raw_text.split())  # Clean spaces

            if not transcript:
                print(f"Skipping empty file: {file_path}")
                continue

            # Keyword extraction across 1, 2, 3 grams
            keywords_dict = {}

            def add_keywords(ngram_range, top_n):
                extracted = kw_model.extract_keywords(
                    transcript,
                    keyphrase_ngram_range=(ngram_range, ngram_range),
                    stop_words='english',
                    top_n=top_n
                )
                for kw, score in extracted:
                    keywords_dict[kw] = round(score, 4)

            add_keywords(1, 15)
            add_keywords(2, 10)
            add_keywords(3, 5)

            # Deduplication: prefer longer phrases
            final_keywords = {}
            for phrase, score in sorted(keywords_dict.items(), key=lambda x: -x[1]):
                if not any(word in final_keywords for word in phrase.split()):
                    final_keywords[phrase] = score

            title = os.path.basename(file_path).replace(".txt", "")
            keywords = ', '.join(final_keywords.keys())

            # Create Prompt & Response
            prompt = f"Generate a transcript for a educational lecture based on the topic {title} focusing on the keywords - {keywords}"
            response = transcript

            out_file.write(json.dumps({"Prompt": prompt, "Response": response}, ensure_ascii=False) + "\n")

print(f"Final unified JSONL file created: {train_jsonl_path}")


Final unified JSONL file created: /content/train3.jsonl


SUMMARY TRANSCRIPT

In [ ]:
!pip install -q transformers

import os
import zipfile
from transformers import AutoTokenizer
from google.colab import files

# Load tokenizer (same as summarizer)
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
token_lengths = []

# Process each uploaded zip file
for zip_filename in uploaded:
    base_name = os.path.splitext(zip_filename)[0]
    extract_path = os.path.join("/content", base_name)

    # Extract contents
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

    # Walk through .txt files and compute token lengths
    for root, _, files_in_dir in os.walk(extract_path):
        for file in sorted(files_in_dir):
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)
                try:
                    with open(file_path, "r", encoding="utf-8") as f:
                        text = f.read()
                    tokens = tokenizer(text, return_tensors="pt", truncation=False)
                    token_len = tokens["input_ids"].shape[1]
                    token_lengths.append(token_len)
                    print(f"{file}: {token_len} tokens")
                except Exception as e:
                    print(f"Error reading {file_path}: {e}")

# Print summary stats
if token_lengths:
    avg_len = sum(token_lengths) / len(token_lengths)
    max_len = max(token_lengths)
    min_len = min(token_lengths)
    print("\nSummary of token lengths:")
    print(f"Average token length: {avg_len:.2f}")
    print(f"Maximum token length: {max_len}")
    print(f"Minimum token length: {min_len}")
else:
    print("No token lengths computed.")


1. The Geometry of Linear Equations.txt: 6043 tokens
10. The Four Fundamental Subspaces.txt: 6971 tokens
11. Matrix Spaces_ Rank 1_ Small World Graphs.txt: 6012 tokens
12. Graphs, Networks, Incidence Matrices.txt: 6761 tokens
13. Quiz 1 Review.txt: 6969 tokens
14. Orthogonal Vectors and Subspaces.txt: 7045 tokens
15. Projections onto Subspaces.txt: 6871 tokens
16. Projection Matrices and Least Squares.txt: 7176 tokens
17. Orthogonal Matrices and Gram-Schmidt.txt: 7348 tokens
18. Properties of Determinants.txt: 7136 tokens
19. Determinant Formulas and Cofactors.txt: 8515 tokens
2. Elimination with Matrices_.txt: 7199 tokens
20. Cramer_s Rule, Inverse Matrix, and Volume.txt: 7980 tokens
21. Eigenvalues and Eigenvectors.txt: 7678 tokens
22. Diagonalization and Powers of A.txt: 7676 tokens
23. Differential Equations and exp(At).txt: 7520 tokens
24. Markov Matrices_ Fourier Series.txt: 7711 tokens
24b. Quiz 2 Review.txt: 7419 tokens
25. Symmetric Matrices and Positive Definiteness.txt: 6374

In [ ]:

def chunk_text(text, max_tokens=950, stride=200):
    tokens = tokenizer(text, return_tensors="pt", truncation=False)["input_ids"][0]
    chunks = []
    start = 0
    while start < len(tokens):
        end = min(start + max_tokens, len(tokens))
        chunk_tokens = tokens[start:end]
        chunk_text = tokenizer.decode(chunk_tokens, skip_special_tokens=True)
        chunks.append(chunk_text)
        if end == len(tokens): break
        start += max_tokens - stride
    return chunks

In [ ]:
from transformers import pipeline, AutoTokenizer
from tqdm import tqdm
import torch

# ========== CONFIG ==========
MAX_TOKENS = 900
CHUNK_OVERLAP = 100
OUTPUT_JSONL = "/content/train.jsonl"

# ========== LOAD MODELS ==========
tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6", device=0 if torch.cuda.is_available() else -1)


# ========== HELPER FUNCTIONS ==========
def chunk_text(text, tokenizer, max_tokens=MAX_TOKENS, overlap=CHUNK_OVERLAP):
    tokens = tokenizer.encode(text)
    chunks = []
    start = 0
    while start < len(tokens):
        end = min(start + max_tokens, len(tokens))
        chunk = tokenizer.decode(tokens[start:end])
        chunks.append(chunk)
        start += max_tokens - overlap
    return chunks

def extract_keywords_from(text, top_n=10):
    return ", ".join([kw for kw, _ in kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 3), stop_words='english', top_n=top_n)])

all_jsonl_lines = []

for zip_filename in uploaded:
    base_name = os.path.splitext(zip_filename)[0]
    extract_path = os.path.join("/content", base_name)

    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

    # Dive into the folder inside if needed
    inner_dirs = os.listdir(extract_path)
    if len(inner_dirs) == 1 and os.path.isdir(os.path.join(extract_path, inner_dirs[0])):
        extract_path = os.path.join(extract_path, inner_dirs[0])

    # ========== PROCESS TEXT FILES ==========
    for file in sorted(os.listdir(extract_path)):
        if file.endswith(".txt"):
            file_path = os.path.join(extract_path, file)
            try:
                with open(file_path, "r", encoding="utf-8") as f:
                    transcript = f.read().strip()

                token_len = len(tokenizer.encode(transcript))
                title = os.path.splitext(file)[0]

                if token_len <= MAX_TOKENS:
                    final_output = transcript
                    keywords = extract_keywords_from(transcript)
                else:
                    chunks = chunk_text(transcript, tokenizer)
                    summaries = []
                    for chunk in chunks:
                        try:
                            summary = summarizer(chunk, max_length=200, min_length=50, do_sample=False)[0]["summary_text"]
                            summaries.append(summary)
                        except Exception as e:
                            print(f"  Skipped one chunk due to summarizer error: {e}")
                    final_output = " ".join(summaries)
                    keywords = extract_keywords_from(final_output)

                json_obj = {
                    "instruction": "generate standard content with the following keywords",
                    "keywords": keywords,
                    "output": final_output
                }

                all_jsonl_lines.append(json.dumps(json_obj))

            except Exception as e:
                print(f"Error processing {file_path}: {e}")

# ========== SAVE TO train.jsonl ==========
with open(OUTPUT_JSONL, "w", encoding="utf-8") as f:
    for line in all_jsonl_lines:
        f.write(line + "\n")

print(f"\n Final dataset saved to: {OUTPUT_JSONL} — Total entries: {len(all_jsonl_lines)}")



Device set to use cuda:0
Token indices sequence length is longer than the specified maximum sequence length for this model (6043 > 1024). Running this sequence through the model will result in indexing errors
Your max_length is set to 200, but your input_length is only 150. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=75)
Your max_length is set to 200, but your input_length is only 188. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=94)
Your max_length is set to 200, but your input_length is only 95. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=47)
Your max_length is set to 200, but your in


 Final dataset saved to: /content/train.jsonl — Total entries: 279
